In [2]:
cascadia_bottom = {
    "x_coord": [],
    "y_coord": [],
    "z_coord": []
}

''' cascadia bottom surfaces
1 231.76194 40.257973 -10 235.61024 50.056671 0 0 0 
2 232.13172 40.057766 -20 236.21259 50.268312 -10 0 0 
3 232.54612 39.841921 -30 236.76067 50.460142 -20 0 0 
4 232.91943 39.728929 -40 237.03881 50.606855 -30 0 0 
5 233.23667 39.627027 -50 237.54666 50.767316 -40 0 0 
6 233.52186 39.537619 -60 237.85776 50.922274 -50 0 0 
7 233.83915 39.431927 -70 238.15881 51.070581 -60 0 0 



 surfaceTag(int(1)) minX(double(231.76194)) minY(double(40.257973)) minZ(-10)
    maxX(235.61024) maxY(50.056671) maxZ(0)
    numPhysicalTags(0) physicalTag(0) ...
    numBoundingCurves(0) curveTag(int; sign encodes orientation) ...
'''

SyntaxError: unterminated string literal (detected at line 2) (447368110.py, line 2)

In [ ]:
# set up

if gmsh.isInitialized() == 0:
        gmsh.initialize()
gmsh.option.setNumber("General.Verbosity", 0)    
gmsh.clear()


# Making a horizontal mesh

# Get bottom of subduction mesh

# Other perimeter control points

# Define a nominal element size
source_el_size = 1

# Use Gmsh to create the mesh
# Gmsh needs geometric elements defined in order of dimension: 
# Define points
# Define lines that connect those points
# Define perimeter as connections of lines
# Define surface from perimeter
# Mesh the surface into triangles



# Define points
for i in range(len(casc_bottom)):
    gmsh.model.geo.addPoint(x, y, z, source_el_size, i)

# Define lines, each referencing the index of the 2 points it connects
for j in range(i+1):
    gmsh.model.geo.addLine(j, j+1, j)
gmsh.model.geo.addLine(j+1, 0, j+1)

# Define curve loop (perimeter), referencing the ordered indices of the lines
gmsh.model.geo.addCurveLoop(list(range(0, j+2)), 1)

# Define surface, referencing the index of the line loop
gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.geo.synchronize()
gmsh.write("mesh.geo_unrolled")
# Generate and write the mesh. Writing a file allows use of meshio
gmsh.model.mesh.generate(2)
gmsh.write("mesh.msh")
gmsh.finalize()

# Read and parse mesh
horiz = dict()
horizobj = meshio.read("horiz.msh")
horiz["points"] = horizobj.points
horiz["verts"] = meshio.CellBlock("triangle", horizobj.get_cells_type("triangle")).data
fig, ax = plt.subplots()
# Draw the fault
ax.triplot(horiz["points"][:, 0], horiz["points"][:, 1], horiz["verts"], linewidth=0.25)
ax.set_aspect("equal")
plt.show()